In [1]:
import numpy as np
import sympy as smp
import scipy as sp
from scipy.integrate import odeint
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from matplotlib import animation
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import PillowWriter

In [2]:
# Defining all variables
m, g, l, a, t, w, b, alpha = smp.symbols('m g l a t w b \alpha')

In [3]:
# Defining theta1 and theta2
the1 , the2 = smp.symbols('\theta_1 \theta_2', cls=smp.Function)

the1= the1(t)
the1_d= smp.diff(the1,t)
the1_dd= smp.diff(the1_d,t)

the2=the2(t)
the2_d= smp.diff(the2,t)
the2_dd= smp.diff(the2_d,t)

In [4]:
# Defining x and y coordinates
x_1, x_2, y_1, y_2= smp.symbols('x_1 x_2 y_1 y_2', cls=smp.Function)

x_1= x_1(t, the1)
y_1= y_1(t, the2)
x_2= x_2(t, the1, the2)
y_2= y_2(t, the1, the2)

In [5]:
# Specific Functional form of x_1,x_2,y_1,y_2

x_1= a*smp.sin(w*t) + l*smp.sin(the1)
y_1= -l*smp.cos(the1)
x_2= a*smp.sin(w*t) + l*smp.sin(the1)+ l*smp.sin(the2)
y_2= -l*smp.cos(the1) - l*smp.cos(the2)

In [6]:
# Defining the velocities
vx1_f= smp.lambdify((the1,the2,l,w,a,the1_d,the2_d,t), smp.diff(x_1,t))
vx2_f= smp.lambdify((the1,the2,l,w,a,the1_d,the2_d,t), smp.diff(x_2,t))
vy1_f= smp.lambdify((the1,the2,l,w,a,the1_d,the2_d,t), smp.diff(y_1,t))
vy2_f= smp.lambdify((the1,the2,l,w,a,the1_d,the2_d,t), smp.diff(y_2,t))

In [7]:
# Define kinetic and potential energy and Langrangian of pendulum
T= 1/2 * m * (smp.diff(x_1,t)**2 + smp.diff(y_1,t)**2 + smp.diff(x_2,t)**2 + smp.diff(y_2,t)**2)
V= m*g*y_1+ m*g*y_2
L= T-V

In [8]:
# Calculation of dissipative force
F = 1/2 * b * (smp.diff(x_1,t)**2 + smp.diff(y_1,t)**2 + smp.diff(x_2,t)**2 + smp.diff(y_2,t)**2)

In [9]:
# Langrangian Equatins
LE1= - smp.diff(L, the1) + smp.diff(smp.diff(L,the1_d), t) + smp.diff(F,the1_d)
LE1= LE1.simplify()

LE2= - smp.diff(L, the2) + smp.diff(smp.diff(L,the2_d), t) + smp.diff(F,the2_d)
LE2= LE2.simplify()


In [10]:
sols= smp.solve([LE1, LE2],(the1_dd, the2_dd), simplify=False, rational= False)

In [11]:
sols[the1_dd]

-a*b*w*cos(t*w)*cos(heta_1(t) - heta_2(t))*cos(heta_2(t))/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m) + 2*a*b*w*cos(t*w)*cos(heta_1(t))/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m) + a*m*w**2*sin(t*w)*cos(heta_1(t) - heta_2(t))*cos(heta_2(t))/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m) - 2*a*m*w**2*sin(t*w)*cos(heta_1(t))/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m) - b*l*cos(heta_1(t) - heta_2(t))**2*Derivative(heta_1(t), t)/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m) + 2*b*l*Derivative(heta_1(t), t)/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m) + 2*g*m*sin(heta_1(t))/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m) - g*m*sin(heta_2(t))*cos(heta_1(t) - heta_2(t))/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m) + l*m*sin(heta_1(t) - heta_2(t))*cos(heta_1(t) - heta_2(t))*Derivative(heta_1(t), t)**2/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m) + l*m*sin(heta_1(t) - heta_2(t))*Derivative(heta_2(t), t)**2/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m)

In [12]:
sols[the2_dd]

-2*a*b*w*cos(t*w)*cos(heta_1(t) - heta_2(t))*cos(heta_1(t))/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m) + 2*a*b*w*cos(t*w)*cos(heta_2(t))/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m) + 2*a*m*w**2*sin(t*w)*cos(heta_1(t) - heta_2(t))*cos(heta_1(t))/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m) - 2*a*m*w**2*sin(t*w)*cos(heta_2(t))/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m) - b*l*cos(heta_1(t) - heta_2(t))**2*Derivative(heta_2(t), t)/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m) + 2*b*l*Derivative(heta_2(t), t)/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m) - 2*g*m*sin(heta_1(t))*cos(heta_1(t) - heta_2(t))/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m) + 2*g*m*sin(heta_2(t))/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m) - l*m*sin(heta_1(t) - heta_2(t))*cos(heta_1(t) - heta_2(t))*Derivative(heta_2(t), t)**2/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m) - 2*l*m*sin(heta_1(t) - heta_2(t))*Derivative(heta_1(t), t)**2/(l*m*cos(heta_1(t) - heta_2(t))**2 - 2*l*m)

In [13]:
# Frequencies that results in resonance
# the1 and the2 are small enough and the1= asin(wt) and the2= a(alpha)sin(wt)

A= LE1.subs([(smp.sin(the1-the2), the1-the2),(smp.cos(the1-the2),1),(smp.cos(the1),1),(smp.cos(the2),1),
             (smp.sin(the1),the1),(smp.sin(the2),the2),(the1, a*smp.sin(w*t)),(the2, a*alpha*smp.sin(w*t))
            ,(b,0)]).doit().series(a,0,2).simplify().removeO()
A

-a*l*m*(lpha*l*w**2 - 2*g + 2*l*w**2 + 2*w**2)*sin(t*w)

In [14]:
B= LE2.subs([(smp.sin(the1-the2), the1-the2),(smp.cos(the1-the2),1),(smp.cos(the1),1),(smp.cos(the2),1),
             (smp.sin(the1),the1),(smp.sin(the2),the2),(the1, a*smp.sin(w*t)),(the2, a*alpha*smp.sin(w*t))
            ,(b,0)]).doit().series(a,0,2).simplify().removeO()
B

-a*l*m*(-lpha*g + lpha*l*w**2 + l*w**2 + w**2)*sin(t*w)

In [15]:
A.args

(-1, a, l, m, lpha*l*w**2 - 2*g + 2*l*w**2 + 2*w**2, sin(t*w))

In [16]:
B.args

(-1, a, l, m, -lpha*g + lpha*l*w**2 + l*w**2 + w**2, sin(t*w))

In [17]:
soln= smp.solve([A.args[4], B.args[4]],(w, alpha))
soln[0][0]

-sqrt(-g*(-2 - sqrt(2*l**2 + 2*l + 1)/l - 1/l)/(l + 1))

In [18]:
soln[0][1]

-sqrt(2*l**2 + 2*l + 1)/l - 1/l

In [19]:
soln[1][0]

sqrt(-g*(-2 - sqrt(2*l**2 + 2*l + 1)/l - 1/l)/(l + 1))

In [20]:
soln[1][1]

-sqrt(2*l**2 + 2*l + 1)/l - 1/l

In [21]:
dz1dt_f = smp.lambdify((t, m, g, w, l, a, the1, the2, the1_d, the2_d), sols[the1_dd])
dthe1dt_f = smp.lambdify(the1_d, the1_d)

dz2dt_f = smp.lambdify((t, m, g, w, l, a, the1, the2, the1_d, the2_d), sols[the2_dd])
dthe2dt_f = smp.lambdify(the2_d, the2_d)

In [22]:
def dSdt(S, t):
    the1, z1, the2, z2 = S
    return [
        dthe1dt_f(z1),
        dz1dt_f(t, m, g, w, l, a, the1, the2, z1, z2),
        dthe2dt_f(z2),
        dz2dt_f(t, m, g, w, l, a, the1, the2, z1, z2),
    ]

In [23]:
help(odeint)

Help on function odeint in module scipy.integrate._odepack_py:

odeint(func, y0, t, args=(), Dfun=None, col_deriv=0, full_output=0, ml=None, mu=None, rtol=None, atol=None, tcrit=None, h0=0.0, hmax=0.0, hmin=0.0, ixpr=0, mxstep=0, mxhnil=0, mxordn=12, mxords=5, printmessg=0, tfirst=False)
    Integrate a system of ordinary differential equations.
    
    .. note:: For new code, use `scipy.integrate.solve_ivp` to solve a
              differential equation.
    
    Solve a system of ordinary differential equations using lsoda from the
    FORTRAN library odepack.
    
    Solves the initial value problem for stiff or non-stiff systems
    of first order ode-s::
    
        dy/dt = func(y, t, ...)  [or func(t, y, ...)]
    
    where y can be a vector.
    
    .. note:: By default, the required order of the first two arguments of
              `func` are in the opposite order of the arguments in the system
              definition function used by the `scipy.integrate.ode` class and
 

In [24]:
t = np.linspace(0, 20, 1000)
t_span=(0,20)
g = 9.81
m=1
l = 1
a = 0.05
w = np.sqrt(g/l)
the1_0 = 0
the2_0 = 0
the1_d_0 = 0
the2_d_0 = 0


In [25]:
help(solve_ivp)

Help on function solve_ivp in module scipy.integrate._ivp.ivp:

solve_ivp(fun, t_span, y0, method='RK45', t_eval=None, dense_output=False, events=None, vectorized=False, args=None, **options)
    Solve an initial value problem for a system of ODEs.
    
    This function numerically integrates a system of ordinary differential
    equations given an initial value::
    
        dy / dt = f(t, y)
        y(t0) = y0
    
    Here t is a 1-D independent variable (time), y(t) is an
    N-D vector-valued function (state), and an N-D
    vector-valued function f(t, y) determines the differential equations.
    The goal is to find y(t) approximately satisfying the differential
    equations, given an initial value y(t0)=y0.
    
    Some of the solvers support integration in the complex domain, but note
    that for stiff ODE solvers, the right-hand side must be
    complex-differentiable (satisfy Cauchy-Riemann equations [11]_).
    To solve a problem in the complex domain, pass y0 with a co

In [26]:
#ans = solve_ivp(dSdt, t_span, y0=[0,0,0,0], method='RK45',t_eval=None, dense_output=False, events=None,
#                vectorized=False, args=None)
ans = odeint(dSdt, y0=[0, 0, 0, 0], t=t)

TypeError: can't convert expression to float